In [11]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import trimesh as tm
import boolean_marching_cubes as bmc

In [12]:
lattice_path = os.path.relpath('../data/voxelized_envelope_toy.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
cube_lattice = envelope_lattice.boolean_marching_cubes()

In [13]:
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# i'm trying to save directly to template folder from rhino but the end results is coming out very different from what i expected 

In [14]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope_toy.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
print(envelope_lattice.shape)
padded_env_arr = np.pad(envelope_lattice, 1, mode="constant", constant_values=False)
padded_minbound = envelope_lattice.minbound - envelope_lattice.unit
envelope_lattice = tg.to_lattice(padded_env_arr, minbound=padded_minbound, unit=envelope_lattice.unit)
print(envelope_lattice.shape)


(2, 3, 3)
(4, 5, 5)


In [25]:
def boolean_marching_cubes(l):
    """This is a polygonization method. It converts the lattice to a boolean lattice and runs a boolean marching cube on the lattice. 

    Returns:
        topogenesis.Lattice: an integer lattice that contains the tile-id at each cell
    """

    # construct the boolean_marching_cubes stencil
    mc_stencil = tg.create_stencil("boolean_marching_cube", 1)
    # retrieve the value of the neighbours
    cell_corners = l.find_neighbours(mc_stencil, order="F")
    # converting volume value (TODO: this needs to become a method of its own)
    volume_flat = l.ravel()
    volume_flat[volume_flat > 0.0] = 1
    volume_flat[volume_flat <= 0.0] = 0

    # replace neighbours by their value in volume
    neighbor_values = volume_flat[cell_corners]

    # computing the cell tile id
    # the powers of 2 in an array
    legend = np.flip(2**np.arange(8))

    # multiply the corner with the power of two, sum them, and reshape to the original volume shape
    tile_id = np.sum(legend * neighbor_values,
                        axis=1).reshape(l.shape)

    # drop the last column, row and page (since cube-grid is 1 less than the voxel grid in every dimension)
    # TODO consider that by implementing the origin attribute in lattice this may have to change
    cube_grid = tile_id[:-1, :-1, :-1]

    # convert the array to lattice
    cube_lattice = tg.to_lattice(
        cube_grid, minbound=l.minbound, unit=l.unit)

    return cube_lattice

In [26]:
cube_lattice = boolean_marching_cubes(envelope_lattice)

[0 0 0]
[[False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False  True]
 [False False False  True False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False  True False  True]
 [False  True False  True False False False  True]
 [False False False  True False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False  True False False]
 [False  True False False False  True False False]
 [False  True False False False False False False]
 [False False False Fal

In [29]:
facade_tiles_path1 = os.path.relpath('../data/bmc/BMC_DUMMY_256')
bmc_facademesh = bmc.marching_cube_mesh(cube_lattice, facade_tiles_path1)

521 472


In [30]:

# initiating the plotter
p = pv.Plotter()

# # adding the meshes
p.add_mesh(tri_to_pv(bmc_facademesh), color='#638275', name="sphere", opacity=1)
# # fast visualization of the lattice
p = envelope_lattice.fast_vis(p)
# p.add_mesh(tri_to_pv(context_mesh), style='surface')
#p.add_mesh(tri_to_pv(bmc_facademesh), style ='surface',opacity=1)
# plotting
p.add_axes()
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(59.21345375285915, 65.69345377193264, 38.58545301662258),
 (23.760000705718994, 30.24000072479248, 3.131999969482422),
 (0.0, 0.0, 1.0)]